In [1]:
#imports
import copy 
import collections
from typing import List,Callable
import heapq
import time
from random import sample

In [2]:
import tensorflow as tf

2024-02-18 12:58:27.819359: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-18 12:58:27.913580: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-18 12:58:27.913666: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-18 12:58:27.917109: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-18 12:58:27.934009: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-18 12:58:27.938669: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
#data structure
class Node:
    def __init__(self,state:List[List[int]],parent,cost:float,heuristic_cost:float=float('inf')):
        self.state = state
        self.parent = parent 
        self.cost = cost
        self.heuristic_cost = heuristic_cost
        self.f = self.heuristic_cost+self.cost
    #operation overriding for less than as we will be storing our nodes in the priority queue
    def __lt__(self, other):
        return self.cost + self.heuristic_cost < other.cost + other.heuristic_cost



In [3]:
#operations
class Methods:
    location = [[-1,-1] for i in range(9)]
    def __init__(self,start:Node,goal:Node):
        self.start = start
        self.goal = goal
    def locationSetter(self) -> None:
        for i in range(3):
            for j in range(3):
                self.location[self.goal.state[i][j]] = [i, j]
    def is_solvable(self,st:Node)->bool:
        # Flatten the state into a one-dimensional list
        flat_state = [n for row in st.state for n in row]
        # Count the number of inversions (pairs of tiles that are out of order)
        inversions = 0
        for i in range(8):
            for j in range(i + 1, 9):
                if flat_state[i] and flat_state[j] and flat_state[i] > flat_state[j]:
                    inversions += 1
        # A state is valid if the number of inversions is even
        return inversions % 2 == 0
    def goalCheck(self,node:Node):
        return self.goal.state==node.state
    def returnNNodes(self,st:Node)->float:
        return st.cost

    # defining the heuristic functions
    def heuristicComputerManhattan(self,node:List[List[int]]):
        cost:int=0
        for i in range(3):
            for j in range(3):
                elem = node[i][j]
                cost+=(abs(self.location[elem][0]-i)+abs(self.location[elem][0]-j))
        return cost
    def heuristicComputerMisplaced(self,node:List[List[int]]):
        cost:int=0
        for i in range(3):
            for j in range(3):
                elem = node[i][j]
                if self.location[elem]!=[i,j]:
                    cost+=1
        return cost
    def HeuristicComputerGaschnigs(self,board:list[list[int]]):
        misplaced_tiles:int = 0
        for i in range(3):
            for j in range(3):
                if board[i][j] != self.goal.state[i][j] and board[i][j] != 0:
                    misplaced_tiles += 1
        return misplaced_tiles
 #corresponding expansion functions
    def expansion(self,st:Node,heuristicComputer:Callable[[List[List[int]]],int])->List[Node]:
        dr = [-1,0,1,0]
        dc = [0,1,0,-1]
        blank_row=-1
        blank_column=-1
        result = list()
        for i in range(3):
            for j in range(3):
                if st.state[i][j]==0:
                    blank_row,blank_column=i,j
                    break

        for i in range(4):
            new_state = copy.deepcopy(st.state)
            new_row=blank_row+dr[i]
            new_column=blank_column+dc[i]
            if 0<=new_row<3 and 0<=new_column<3:
                new_state[new_row][new_column],new_state[blank_row][blank_column] = new_state[blank_row][blank_column],new_state[new_row][new_column]
                newNode = Node(new_state,st,st.cost+1,heuristicComputer(new_state))
                if self.is_solvable(newNode):
                    result.append(newNode)
        return result

    def AStarSearch(self,heuristicComputer:Callable[[List[List[int]]],int])->float:
        pq = list()
        visited = collections.defaultdict(bool)
        heapq.heappush(pq,(self.start.cost+self.start.heuristic_cost,self.start))
        visited[self.start] = True
        while pq:
            f_node,node = heapq.heappop(pq)
            if self.is_solvable(node) is False:
                return -1
            if self.goalCheck(node) and f_node<=pq[0][0]:
                return self.returnNNodes(node)
            neighbours:List[Node] = self.expansion(node,heuristicComputer)
            for neighbor in neighbours:
                if neighbor not in visited :
                    visited[neighbor] = True
                    heapq.heappush(pq,(neighbor.cost+neighbor.heuristic_cost,neighbor))
        return -1
    def idAStarSearch(self, heuristicComputer: Callable[[List[List[int]]], int]) -> float:
        def search(node, g, bound):
            f = g + heuristicComputer(node.state)
            if f > bound:
                return f
            if self.goalCheck(node):
                return 'FOUND'
            min_cost = float('inf')
            neighbors = self.expansion(node, heuristicComputer)
            for neighbor in neighbors:
                t = search(neighbor, g + 1, bound)
                if t == 'FOUND':
                    return 'FOUND'
                if t < min_cost:
                    min_cost = t
            return min_cost
        
        bound = heuristicComputer(self.start.state)
        while True:
            result = search(self.start, self.start.cost, bound)
            if result == 'FOUND':
                return self.returnNNodes(self.start)
            if result == float('inf'):
                return -1
            bound = result
    def weightedAStar(self, heuristicComputer: Callable[[List[List[int]]], int], weight: float=5) -> float:
        pq = []  # Priority queue
        visited = collections.defaultdict(bool)
        heapq.heappush(pq, (self.start.cost + weight * heuristicComputer(self.start.state), self.start))
        visited[self.start] = True
        
        while pq:
            f_node, node = heapq.heappop(pq)
            
            if self.is_solvable(node) is False:
                return -1
            
            if self.goalCheck(node):
                return self.returnNNodes(node)
            
            neighbors = self.expansion(node, heuristicComputer)
            for neighbor in neighbors:
                if neighbor not in visited:
                    visited[neighbor] = True
                    heapq.heappush(pq, (neighbor.cost + weight * heuristicComputer(neighbor.state), neighbor))
        
        return -1
    def RecursiveBestFirst(self,heuristicComputer:Callable[[List[List[int]]],int]):
        answer:int=0
        def RBFS(node:Node,f_limit:float):
            if self.goalCheck(node):
                answer = self.returnNNodes(node)
                return None,float('inf')
            successors:list[Node] = self.expansion(node,heuristicComputer)
            if not successors:
                return (None,float('inf'))
            for child in successors:
                child.f = max((child.cost+child.heuristic_cost),node.f)
            while 1:
                sorted_succ = sorted(successors)
                best:Node = sorted_succ[0]
                if best.f>f_limit:
                    return (None,best.f)
                next_best:Node = sorted_succ[1]
                parser:float = min(f_limit,next_best.f)
                result,best.f = RBFS(best,parser)
                if result is not None:
                    return (result,best.f)
        RBFS(self.start,float('inf'))
        return answer



In [4]:
#random matrix generator
def matrixGenerator():
    matrix = sample(list(range(0,9)),9)
    return [matrix[0:3],matrix[3:6],matrix[6:]]

In [5]:
# main 
if __name__=="__main__":
    start:Node = Node([[1,8,2],[0,4,3],[7,6,5]],None,0)
    goal:Node = Node([[1,2,3],[4,5,6],[7,8,0]],None,float('inf'),0)
    operation = Methods(start,goal)
    operation.locationSetter()
    operation.start.heuristic_cost =  operation.heuristicComputerManhattan([[1,8,2],[0,4,3],[7,6,5]])
    for nameAlgo,algorithms in [(" ",None),("A*\t",operation.AStarSearch),("IDA*\t",operation.idAStarSearch),("weightedA*\t",operation.weightedAStar),("RBFS\t",operation.RecursiveBestFirst)]:
        print(f"{nameAlgo}->",end=" ")
        for nameComputer,heuristicComputer in [("Gaschings",operation.HeuristicComputerGaschnigs),("Manhattan",operation.heuristicComputerManhattan),("Misplaced",operation.heuristicComputerMisplaced)]:
            if algorithms is None:
                print(f"\t{nameComputer}",end=" ")
            else:
                start_time = time.time()
                res = algorithms(heuristicComputer)
                end_time = time.time()
                print(f"{res},{end_time-start_time} seconds",end=" ")
        print()

 -> 	Gaschings 	Manhattan 	Misplaced 
A*	-> 9,0.01613473892211914 seconds 11,0.016236305236816406 seconds 9,0.008007287979125977 seconds 
IDA*	-> 0,0.00800466537475586 seconds 0,0.008005619049072266 seconds 0,0.016004562377929688 seconds 
weightedA*	-> 9,0.0 seconds 11,0.0807504653930664 seconds 9,0.0 seconds 
RBFS	-> 


KeyboardInterrupt

